In [1]:
from pyspark.sql import SparkSession

In [2]:
spark=SparkSession.builder.master('local[*]').getOrCreate()

In [3]:
from pyspark.sql.functions import to_timestamp, col, lit
rc=spark.read.csv('reported-crimes.csv',header=True).withColumn('Date',to_timestamp(col('Date'),'MM/dd/yyyy hh:mm:ss a')).filter(col('Date')<=lit('2018-11-11'))

In [4]:
rc.columns

['ID',
 'Case Number',
 'Date',
 'Block',
 'IUCR',
 'Primary Type',
 'Description',
 'Location Description',
 'Arrest',
 'Domestic',
 'Beat',
 'District',
 'Ward',
 'Community Area',
 'FBI Code',
 'X Coordinate',
 'Y Coordinate',
 'Year',
 'Updated On',
 'Latitude',
 'Longitude',
 'Location']

In [5]:
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, DoubleType, BooleanType, TimestampType

In [6]:
labels=[
    ('ID',StringType()),
    ('Case Number',StringType()),
    ('Date',TimestampType()),
    ('Block',StringType()),
    ('IUCR',StringType()),
    ('Primary Type',StringType()),
    ('Description',StringType()),
    ('Location Description',StringType()),
    ('Arrest',StringType()),
    ('Domestic',BooleanType()),
    ('Beat',StringType()),
    ('District',StringType()),
    ('Ward',StringType()),
    ('Community Area',StringType()),
    ('FBI Code',StringType()),
    ('X Coordinate',StringType()),
    ('Y Coordinate',StringType()),
    ('Year',IntegerType()),
    ('Updated On',TimestampType()),
    ('Latitude',DoubleType()),
    ('Longitude',DoubleType()),
    ('Location',StringType())
]

In [7]:
schema=StructType([StructField (x[0],x[1],True) for x in labels])

In [8]:
rc.show(5)

+--------+-----------+-------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|      ID|Case Number|               Date|               Block|IUCR|      Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|Latitude|Longitude|Location|
+--------+-----------+-------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|11037294|   JA371270|2015-03-18 12:00:00|   0000X W WACKER DR|1153|DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|                BANK| false|   false|0111|     001|  42|            32|      11|     

In [9]:
##Accessing columns

In [10]:
rc.select('IUCR').show(5)

+----+
|IUCR|
+----+
|1153|
|1153|
|1153|
|0810|
|1153|
+----+
only showing top 5 rows



In [11]:
rc.select('Case Number','Date','Arrest').show(4)

+-----------+-------------------+------+
|Case Number|               Date|Arrest|
+-----------+-------------------+------+
|   JA371270|2015-03-18 12:00:00| false|
|   JC212333|2016-05-01 00:25:00| false|
|   JC212935|2014-06-01 00:01:00| false|
|   JC213529|2018-09-01 00:01:00| false|
+-----------+-------------------+------+
only showing top 4 rows



In [12]:
rc1=rc.withColumn('one',lit(1))

In [13]:
rc2=rc1.drop('IUCR')

In [14]:
rc2.limit(5).show()

+--------+-----------+-------------------+--------------------+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+---+
|      ID|Case Number|               Date|               Block|      Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|Latitude|Longitude|Location|one|
+--------+-----------+-------------------+--------------------+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+---+
|11037294|   JA371270|2015-03-18 12:00:00|   0000X W WACKER DR|DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|                BANK| false|   false|0111|     001|  42|            32|      11|        null|

In [15]:
rc2.show(5)

+--------+-----------+-------------------+--------------------+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+---+
|      ID|Case Number|               Date|               Block|      Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|Latitude|Longitude|Location|one|
+--------+-----------+-------------------+--------------------+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+---+
|11037294|   JA371270|2015-03-18 12:00:00|   0000X W WACKER DR|DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|                BANK| false|   false|0111|     001|  42|            32|      11|        null|

In [16]:
rc2.limit(5).show()

+--------+-----------+-------------------+--------------------+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+---+
|      ID|Case Number|               Date|               Block|      Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|Latitude|Longitude|Location|one|
+--------+-----------+-------------------+--------------------+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+---+
|11037294|   JA371270|2015-03-18 12:00:00|   0000X W WACKER DR|DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|                BANK| false|   false|0111|     001|  42|            32|      11|        null|

In [17]:
one_day=spark.read.csv('reported-crimes.csv',header=True).withColumn('Date',to_timestamp(col('Date'),'MM/dd/yyyy hh:mm:ss a')).filter(col('Date')==lit('2018-11-12'))

In [18]:
rc_combined=rc.union(one_day)

In [19]:
rc_combined.groupBy(col('Location Description')).count().orderBy('count',ascending=False).show(3)

+--------------------+-------+
|Location Description|  count|
+--------------------+-------+
|              STREET|1770635|
|           RESIDENCE|1146271|
|           APARTMENT| 699235|
+--------------------+-------+
only showing top 3 rows



In [20]:
rc_combined.select('Arrest').distinct().show()

+------+
|Arrest|
+------+
| false|
|  true|
+------+



In [21]:
# %%time
# for i in range(0,3):
#     val=(rc_combined.filter(col('Arrest')=='true').count()*1.0)/rc_combined.count()

In [22]:
rc_combined.count()

6756755

In [23]:
# %%time
# for i in range(0,3):
#     val=(rc_combined.filter(col('Arrest')==True).count()*1.0)/rc_combined.count()

In [24]:
rc_combined.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: string (nullable = true)
 |-- Domestic: string (nullable = true)
 |-- Beat: string (nullable = true)
 |-- District: string (nullable = true)
 |-- Ward: string (nullable = true)
 |-- Community Area: string (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: string (nullable = true)
 |-- Y Coordinate: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Updated On: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Location: string (nullable = true)



In [25]:
rc_combined.show(5,truncate=False)

+--------+-----------+-------------------+---------------------+----+------------------+-----------------------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+----------------------+--------+---------+--------+
|ID      |Case Number|Date               |Block                |IUCR|Primary Type      |Description                        |Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|Updated On            |Latitude|Longitude|Location|
+--------+-----------+-------------------+---------------------+----+------------------+-----------------------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+----------------------+--------+---------+--------+
|11037294|JA371270   |2015-03-18 12:00:00|0000X W WACKER DR    |1153|DECEPTIVE PRACTICE|FINANCIAL IDENTITY THEFT OVER $ 300|BANK           

In [26]:
ps=spark.read.csv('Police_Stations.csv',header=True)

In [27]:
ps.show(5)

+------------+-------------+-------------------+-------+-----+-----+--------------------+------------+------------+------------+------------+------------+-----------+------------+--------------------+
|    DISTRICT|DISTRICT NAME|            ADDRESS|   CITY|STATE|  ZIP|             WEBSITE|       PHONE|         FAX|         TTY|X COORDINATE|Y COORDINATE|   LATITUDE|   LONGITUDE|            LOCATION|
+------------+-------------+-------------------+-------+-----+-----+--------------------+------------+------------+------------+------------+------------+-----------+------------+--------------------+
|Headquarters| Headquarters|3510 S Michigan Ave|Chicago|   IL|60653|http://home.chica...|        null|        null|        null| 1177731.401| 1881697.404|41.83070169|-87.62339535|(41.8307016873, -...|
|          18|   Near North| 1160 N Larrabee St|Chicago|   IL|60610|http://home.chica...|312-742-5870|312-742-5771|312-742-5773| 1172080.029| 1908086.527|41.90324165|-87.64335214|(41.9032416531, -

In [28]:
from pyspark.sql.functions import dayofweek

In [29]:
rc_combined_weekday=rc_combined.withColumn('day_of_week',dayofweek(col('Date')))

In [44]:
dow=spark.read.csv('week_day.csv',header=True)

In [45]:
rc_grouped_daily=rc_combined_weekday.groupBy('day_of_week').count()

In [49]:
rc_grouped_daily.join(dow,rc_grouped_daily.day_of_week==dow.day_number).select('day_name','count').show()


+---------+-------+
| day_name|  count|
+---------+-------+
|   Sunday| 912447|
|   Friday|1017464|
|  Tuesday| 968538|
| Thursday| 965103|
|Wednesday| 974379|
| Saturday| 965584|
|   Monday| 953240|
+---------+-------+



In [47]:
joined_df.select('day_name','count').show()

+---------+-------+
| day_name|  count|
+---------+-------+
|   Sunday| 912447|
|   Friday|1017464|
|  Tuesday| 968538|
| Thursday| 965103|
|Wednesday| 974379|
| Saturday| 965584|
|   Monday| 953240|
+---------+-------+

